In [76]:
import configparser
import numpy as np
import pandas as pd
import tasks.models as m

from sqlalchemy import create_engine, func, distinct
from sqlalchemy.orm import Session
from sqlalchemy import tuple_
import redis

In [77]:
config = configparser.ConfigParser()
config.read("config/config.ini")

engine = create_engine(config["DB"].get("db_url"))
session = Session(engine)

r = redis.Redis(
    host=config["REDIS"].get("host"),
    port=config["REDIS"].get("port"),
    db=config["REDIS"].get("db"),
)

In [78]:
s = (
    session.query(m.Redditor_Subreddit_Comment.redditor_id)
    .group_by(m.Redditor_Subreddit_Comment.redditor_id)
    .having(func.count(m.Redditor_Subreddit_Comment.subreddit_id.distinct()) > 2)
    .subquery()
)
redditor_subreddit_comment = np.array(
    session.query(
        m.Redditor_Subreddit_Comment.redditor_id, m.Subreddit.name, m.Redditor.name
    )
    .distinct(m.Redditor_Subreddit_Comment.redditor_id, m.Subreddit.name)
    .filter(m.Redditor_Subreddit_Comment.redditor_id.in_(s))
    .join(m.Subreddit, m.Redditor_Subreddit_Comment.subreddit_id == m.Subreddit.id)
    .join(m.Redditor, m.Redditor_Subreddit_Comment.redditor_id == m.Redditor.id)
    .all()
).T
df = pd.DataFrame(
    {
        "user_id": redditor_subreddit_comment[0],
        "subreddit_id": redditor_subreddit_comment[1],
        "username": redditor_subreddit_comment[2],
    }
)

In [79]:
user_ids = np.unique(df['user_id'])

In [80]:
def compare_recommendation_and_interaction(user_id):
    if type(user_id) != list:
        user_id = [user_id]
    for u in user_id:
        print(f"user: {df[df['user_id']==u]['username'].iloc[0]}") 
        print(f"commented on {np.unique(df[df['user_id']==u]['subreddit_id'])}")
        print(f"recommended {list(map(lambda x: x.decode(), r.lrange(u, 0, -1)))}")
        print("")


In [81]:
compare_recommendation_and_interaction(user_ids[0])

user: tokenflip408619
commented on ['MTB' 'crossfit' 'guns']
recommended ['texas', 'vinyl', 'Austin', 'Seattle', 'Coffee']



In [82]:
compare_recommendation_and_interaction(user_ids[1])

user: ImaginaryTutor
commented on ['SelfAwarewolves' 'lgbt' 'traaaaaaannnnnnnnnns']
recommended ['actuallesbians', 'AreTheStraightsOK', 'TheRightCantMeme', 'bisexual', 'ToiletPaperUSA']



In [83]:
compare_recommendation_and_interaction(user_ids[2])

user: tarkin1980
commented on ['CitiesSkylines' 'PropagandaPosters' 'Warhammer40k' 'hoi4']
recommended ['eu4', 'Warthunder', 'vexillology', 'TankPorn', 'paradoxplaza']



In [84]:
compare_recommendation_and_interaction(user_ids[3])

user: Norokk
commented on ['RedDeadOnline' 'TrueCrime' 'forza']
recommended ['serialkillers', 'TrueCrimeDiscussion', 'BeautyGuruChatter', 'FragileWhiteRedditor', 'CitiesSkylines']



In [85]:
compare_recommendation_and_interaction(user_ids[4])

user: DerekB52
commented on ['linuxquestions' 'makinghiphop' 'unixporn']
recommended ['linux', 'linuxmasterrace', 'FL_Studio', 'trapproduction', 'archlinux']



In [86]:
compare_recommendation_and_interaction(user_ids[5])

user: phillyphillster34
commented on ['Smite' 'TechNewsToday' 'Windows10' 'microgrowery' 'windows']
recommended ['sysadmin', 'feedthebeast', 'homelab', 'typography', 'DataHoarder']



In [87]:
compare_recommendation_and_interaction(user_ids[10])

user: TextMekks
commented on ['AskEngineers' 'MortalKombat' 'pussypassdenied']
recommended ['engineering', 'EngineeringStudents', 'careerguidance', 'ActualPublicFreakouts', 'ThatLookedExpensive']



In [88]:
compare_recommendation_and_interaction(user_ids[110])

user: SirKronik
commented on ['2007scape' 'MMORPG' 'SquaredCircle']
recommended ['ffxiv', 'hometheater', 'runescape', 'Guildwars2', 'chicago']



In [89]:
compare_recommendation_and_interaction(user_ids[1000])

user: waywardhero
commented on ['PoliticalCompassMemes' 'knives' 'roosterteeth']
recommended ['EDC', 'FallGuysGame', 'guns', 'ffxiv', 'masseffect']



In [90]:
# compare_recommendation_and_interaction('10rndb')
compare_recommendation_and_interaction(list(np.unique(df[df['subreddit_id']=='learnpython']['user_id'])))

user: xenaprincesswarlord
commented on ['HowToHack' 'TechNewsToday' 'UKPersonalFinance' 'learnpython']
recommended ['feedthebeast', 'cybersecurity', 'datascience', 'finance', 'web_design']

user: SweeTLemonS_TPR
commented on ['Coffee' 'LeopardsAteMyFace' 'Rabbits' 'cscareerquestions'
 'cybersecurity' 'learnpython']
recommended ['tea', 'philadelphia', 'HowToHack', 'finance', 'whiskey']

user: MeanTemperature
commented on ['ich_iel' 'learnpython' 'productivity']
recommended ['selfimprovement', 'de', 'webdev', 'web_design', 'startups']

user: wsppan
commented on ['The_Mueller' 'computerscience' 'learnpython']
recommended ['cscareerquestions', 'feedthebeast', 'Fuckthealtright', 'HomeNetworking', 'HalfLife']

user: funKmaster_tittyBoi
commented on ['TheRightCantMeme' 'learnpython' 'sanfrancisco']
recommended ['bayarea', 'finance', 'datascience', 'CrackheadCraigslist', 'askgaybros']

user: Doormatty
commented on ['arduino' 'learnpython' 'vancouver']
recommended ['techsupportgore', 'PersonalF